In [4]:
import numpy as np
import librosa
import tensorflow.compat.v1 as tf
import tf_slim.layers as _layers
import os
import matplotlib.pyplot as plt
tf.disable_eager_execution()

def audio_read(f):
    y, sr = librosa.core.load(f, sr=22050)
    d = librosa.core.get_duration(y=y, sr=sr)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=512, n_mels=128)
    S = np.transpose(np.log(1+10000*S))
    S = np.expand_dims(S, axis=0)
    return y, S, int(d)

def chunk(incoming, n_chunk):
    input_length = incoming.shape[1]
    chunk_length = input_length // n_chunk
    outputs = []
    for i in range(incoming.shape[0]):
        for j in range(n_chunk):
            outputs.append(incoming[i, j*chunk_length:(j+1)*chunk_length, :])
    outputs = np.array(outputs)
    return outputs

def positional_encoding(batch_size, n_pos, d_pos):
    # keep dim 0 for padding token position encoding zero vector
    position_enc = np.array([
        [pos / np.power(10000, 2 * (j // 2) / d_pos) for j in range(d_pos)]
        if pos != 0 else np.zeros(d_pos) for pos in range(n_pos)])

    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2]) # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2]) # dim 2i+1
    position_enc = np.tile(position_enc, [batch_size, 1, 1])
    return position_enc

class MusicHighlighter(object):
    def __init__(self):
        self.dim_feature = 64

        self.bn_params = {'is_training': True,
                        'center': True, 'scale': True,
                        'updates_collections': None}
        
        self.x = tf.placeholder(tf.float32, shape=[None, None, 128])
        self.y = tf.placeholder(tf.float32, shape=[None, 174])
        self.pos_enc = tf.placeholder(tf.float32, shape=[None, None, self.dim_feature*4])
        self.num_chunk = tf.placeholder(tf.int32)
        self.build_model()
        self.saver = tf.train.Saver(tf.global_variables())
    
    def conv(self, inputs, filters, kernel, stride):
        dim = inputs.get_shape().as_list()[-2]
        return _layers.conv2d(inputs, filters, 
                                        [kernel, dim], [stride, dim],
                                        activation_fn=tf.nn.relu,
                                        normalizer_fn=_layers.batch_norm,
                                        normalizer_params=self.bn_params)

    def fc(self, inputs, num_units, act=tf.nn.relu):
        return _layers.fully_connected(inputs, num_units,
                                                 activation_fn=act,
                                                 normalizer_fn=_layers.batch_norm,
                                                 normalizer_params=self.bn_params)

    def attention(self, inputs, dim):
        outputs = self.fc(inputs, dim, act=tf.nn.tanh)
        outputs = self.fc(outputs, 1, act=None)
        attn_score = tf.nn.softmax(outputs, dim=1)
        return attn_score

    def build_model(self):
        net = tf.expand_dims(self.x, axis=3)
        net = self.conv(net, self.dim_feature, 3, 2)
        net = self.conv(net, self.dim_feature*2, 4, 2)
        net = self.conv(net, self.dim_feature*4, 4, 2)

        net = tf.squeeze(tf.reduce_max(net, axis=1), axis=1)
        
        net = tf.reshape(net, [1, self.num_chunk, self.dim_feature*4])

        attn_net = net + self.pos_enc
        attn_net = self.fc(attn_net, self.dim_feature*4)
        attn_net = self.fc(attn_net, self.dim_feature*4)
        self.attn_score = self.attention(attn_net, self.dim_feature*4)

        net = self.fc(net, 1024)
        chunk_predictions = self.fc(net, 174, act=tf.nn.softmax)
        self.overall_predictions = tf.squeeze(tf.matmul(self.attn_score, chunk_predictions, transpose_a=True), axis=1)

        # overall_predictions = tf.clip_by_value(overall_predictions, 1e-10, 1.0)

        self.loss = tf.reduce_mean(-tf.reduce_sum(self.y * tf.log(self.overall_predictions), axis=1))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(self.loss)


def load_song_names(file_path, base_directory):
    with open(file_path, 'r') as f:
        song_names = [line.strip() for line in f.readlines()]
    
    song_paths = [os.path.join(base_directory, name + '.mp3') for name in song_names]
    
    return song_paths

def load_hard_annotations(file_path):
    return np.loadtxt(file_path, delimiter=',')

def train_model(model, song_paths, labels, epochs=5):

    loss_values = []  
    accuracies = []   

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver()

        for epoch in range(epochs):
            epoch_loss = 0
            correct_predictions = 0
            count = 0
            for i in range(len(song_paths)):
                song_path = song_paths[i]

                if not os.path.exists(song_path):
                    continue 

                label = labels[i]
                label = np.expand_dims(label, axis=0)
                audio, spectrogram, duration = audio_read(song_path)
                if duration < 30:
                    continue

                count += 1
                print(count)

                n_chunk, remainder = np.divmod(duration, 3)

                chunk_spec = chunk(spectrogram, n_chunk)

                pos = positional_encoding(batch_size=1, n_pos=n_chunk, d_pos=model.dim_feature * 4)

                n_chunk = n_chunk.astype('int')
                chunk_spec = chunk_spec.astype('float')
                pos = pos.astype('float')

                # Huấn luyện mô hình với bài hát hiện tại
                feed_dict = {
                    model.x: chunk_spec,
                    model.y: label,
                    model.pos_enc: pos,
                    model.num_chunk: n_chunk
                }

                _, loss_value = sess.run([model.optimizer, model.loss], feed_dict=feed_dict)
                epoch_loss += loss_value

                predicted_probabilities = sess.run(tf.nn.softmax(model.overall_predictions), feed_dict=feed_dict)

                predicted_index = np.argmax(predicted_probabilities, axis=1)[0] 

                if label[0, predicted_index] == 1: 
                    correct_predictions += 1

            avg_loss = epoch_loss / count
            accuracy = correct_predictions / count
            loss_values.append(avg_loss)
            accuracies.append(accuracy)
            
            print(f'Epoch {epoch + 1}, Average Loss: {avg_loss}, Accuracy: {accuracy}')
        model.saver.save(sess, 'model/model')
        print("Save success")

    # Trực quan hóa loss và accuracy
    plt.figure(figsize=(12, 5))

    # Đồ thị loss
    plt.subplot(1, 2, 1)
    plt.plot(loss_values, label='Loss', color='blue')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Đồ thị accuracy
    plt.subplot(1, 2, 2)
    plt.plot(accuracies, label='Accuracy', color='green')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

if __name__ == '__main__':
    base_directory = 'CAL500_32kps'
    
    song_paths = load_song_names('songNames.txt', base_directory)
   
    labels = load_hard_annotations('hardAnnotations.txt')

    model = MusicHighlighter()

    train_model(model, song_paths, labels, epochs=1)


c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1697: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242


c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1728
  warnings.warn(


243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
